<a href="https://colab.research.google.com/github/xuhu357/DataAnalysis/blob/master/ch07_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A4%80%EB%B9%84%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 데이터 다듬기, 변형, 병합

데이터 분석과 모델링 작업


*   데이터를 불러오고, 다듬고, 변형하고 재정렬하는 준비과정에 많은 시간 소요
*   파이썬 표준라이브러리와 pandas를 사용하면, 큰 어려움 없이 데이터를 원하는 형태로 가공 가능.
* pandas는 이런 작업을 유연하고 빠른 고수준의 알고리즘과 처리 기능을 제공.



### 데이터 합치기

pandas 객체에 저장된 데이터는 여러 내장함수를 이용해서 합칠수 있다.
* pandas.merge는 하나 이상의 키를 기준으로 DataFrame의 로우를 합친다.
SQL이나, 다른 관계형 데이터베이스의 join 연산과 유사하다.
* pandas.concat는 하나의 축을 따라 객체를 이어붙인다.
* combine_first 인스턴스 메소드는 두 객체를 포개서 한객체에서 누락된 데이터를 다른 객체에 있는 값으로 채울 수 있다. 

Merge 나 Join연산은 관계형 데이터 베이스의 핵심적인 연산으로 키를 하나 이상 사용해서 데이터 집합의 로우를 합친다. 

pandas의 merge 함수를 이용해서 이런 알고리즘을 데이터에 적용가능하다. 

In [0]:
from pandas import Series, DataFrame

import pandas as pd
import numpy as np

In [0]:
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 
                'data1': range(7)})

In [0]:
df2 = DataFrame({'key': ['a', 'b', 'd'], 
                'data2': range(3)})

In [4]:
df1

data1 key
0      0   b
1      1   b
2      2   a
3      3   c
4      4   a
5      5   a
6      6   b

In [5]:
df2

data2 key
0      0   a
1      1   b
2      2   d

In [6]:
pd.merge(df1, df2)

data1 key  data2
0      0   b      1
1      1   b      1
2      6   b      1
3      2   a      0
4      4   a      0
5      5   a      0

위의 예제에서는 merge 할때, 어떤 칼럼을 병합할 것인지 명시하지 않았는데, merge 함수는 겹치는 칼럼이름을 키로 사용한다. 

하지만, 명시적으로 지정하는 습관을 들이는게 좋음.

In [7]:
pd.merge(df1, df2, on='key')

data1 key  data2
0      0   b      1
1      1   b      1
2      6   b      1
3      2   a      0
4      4   a      0
5      5   a      0

만약, 두 객체에 공통되는 칼럼이 하나라도 없으면, 따로 지정해주면 된다. 

In [0]:
df3 = DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 
                'data1':range(7)})

df4 = DataFrame({'rkey': ['a', 'b', 'd'],
                'data2':range(3)})

In [9]:
df3

data1 lkey
0      0    b
1      1    b
2      2    a
3      3    c
4      4    a
5      5    a
6      6    b

In [10]:
df4

data2 rkey
0      0    a
1      1    b
2      2    d

In [12]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

data1 lkey  data2 rkey
0      0    b      1    b
1      1    b      1    b
2      6    b      1    b
3      2    a      0    a
4      4    a      0    a
5      5    a      0    a

Merge 함수는 기본적으로 내부조인(inner join)을 수행하여 교집합을 반환.

* how 인자로 'left', 'right', 'outer'을 넘겨서 각각, 왼쪽 우선 외부조인, 오른쪽 우선 외부조인, 완전 외부 조인을 수행가능

* 완전 외부조인은 합집합인 결과를 반환하고, 왼쪽 우선, 오른쪽 우선 외부조인은 각각 왼쪽, 오른쪽의 모든 로우를 포함하는 결과를 반환.

예제를 보면, 조금 더 쉽게 이해가능함.

In [13]:
df1

data1 key
0      0   b
1      1   b
2      2   a
3      3   c
4      4   a
5      5   a
6      6   b

In [14]:
df2

data2 key
0      0   a
1      1   b
2      2   d

df1, df2에 대해서 outer join을 해보면,

In [15]:
pd.merge(df1, df2, how='outer')

data1 key  data2
0    0.0   b    1.0
1    1.0   b    1.0
2    6.0   b    1.0
3    2.0   a    0.0
4    4.0   a    0.0
5    5.0   a    0.0
6    3.0   c    NaN
7    NaN   d    2.0

**다 대 다 병합**은 정의되어 있긴 하지만, 직관적이지 않다. 
일단 예를 보고 이해하도록 하자. 


In [0]:
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'], 
                'data1':range(6)})

df2 = DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                'data2':range(5)})

In [17]:
df1

data1 key
0      0   b
1      1   b
2      2   a
3      3   c
4      4   a
5      5   b

In [18]:
df2

data2 key
0      0   a
1      1   b
2      2   a
3      3   b
4      4   d

In [19]:
pd.merge(df1,df2, on='key', how='left')

data1 key  data2
0       0   b    1.0
1       0   b    3.0
2       1   b    1.0
3       1   b    3.0
4       2   a    0.0
5       2   a    2.0
6       3   c    NaN
7       4   a    0.0
8       4   a    2.0
9       5   b    1.0
10      5   b    3.0

보다 싶이, left join은 왼쪽 모든 로우를 포함하는 결과를 반환한다.

---

아래에는 inner join에 대해서 알아보자.

In [20]:
pd.merge(df1, df2, how='inner') # 교집합

data1 key  data2
0      0   b      1
1      0   b      3
2      1   b      1
3      1   b      3
4      5   b      1
5      5   b      3
6      2   a      0
7      2   a      2
8      4   a      0
9      4   a      2

**여러개의 키**를 병합하려면, 칼럼 이름이 들어간 리스트를 넘기면 된다. 

In [0]:
left = DataFrame(
    {
        'key1': ['foo', 'foo', 'bar'],
        'key2': ['one', 'two', 'one'],
        'lval': [1, 2, 3]
    }
)

In [0]:
right = DataFrame(
    {
        'key1': ['foo', 'foo', 'bar', 'bar'],
        'key2': ['one', 'one', 'one', 'two'],
        'rval': [4, 5, 6, 7]
    }
)

In [23]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

key1 key2  lval  rval
0  foo  one   1.0   4.0
1  foo  one   1.0   5.0
2  foo  two   2.0   NaN
3  bar  one   3.0   6.0
4  bar  two   NaN   7.0

머지 연산에서 마지막 고려해야 할 사항은 겹치는 칼럼 이름에 대한 처리이다. 

축의 이름을 변경해서, 수동으로 칼럼 이름을 겹치게 할 수도 있고, merge 함수에 있는 suffixes인자를 통해 두 DataFrame의 겹치는 칼럼 이름 뒤에 붙일 문자열을 지정해 줄 수도 있다. 

뭔말일까? 예제를 보면 알게 됨.

In [24]:
pd.merge(left, right, on='key1')

key1 key2_x  lval key2_y  rval
0  foo    one     1    one     4
1  foo    one     1    one     5
2  foo    two     2    one     4
3  foo    two     2    one     5
4  bar    one     3    one     6
5  bar    one     3    two     7

In [0]:
pd.merge(left, right, on='key1', suffixes=('_left', 'right'))

key1 key2_left  lval key2right  rval
0  foo       one     1       one     4
1  foo       one     1       one     5
2  foo       two     2       one     4
3  foo       two     2       one     5
4  bar       one     3       one     6
5  bar       one     3       two     7

Merge 함수에 대해서 잠깐 정리를 해보면, 

* left: 머지하려는 DataFrame 중 왼족에 위치한 DataFrame
* right: 머지하려는 DataFrame중 오른쪽에 위치한 DataFrame
* how: 조인 방법, inner, outer, left, right, 기본값은 inner.
* on: 조인하려는 row 이름. 두 DataFrame에 모두 있는 이름이어야 한다.
* left_on: 조인 키로 사용할 left DataFrame
* right_on: 조인 키로 사용할 right DataFrame
* left_index: 조인 키로 사용할 left DataFrame의 색인 로우
* right_index: 조인 키로 사용할 right DataFrame의 색인 로우
* sort: 조인 키에 따라 병합된 데이터를 사전 순으로 정렬, 기본 값은 True, 대용량 데이터의 경우, False 라면 성능상의 이득을 얻을 수도 있음.
* suffixes: 칼럼 이름이 겹칠 경우, 각 칼럼 이름 뒤에 붙일 문자열의 튜플, 기본 값은 (_x, _y) 만약  data 라는 칼럼 이름이 양쪽 DataFrame에 모두 있을 경우, 결과에서는 data_x, data_y로 나타남. 
* copy: False일 경우, 예외적인 경ㅇ우에 결과로 데이터가 복사되지 않도록 한다. 기본 값은 항상 복사가 이루어 진다.